In [451]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from urllib.parse import quote_plus
from bs4 import BeautifulSoup, Comment, Doctype
import re
import time 


# Cargar variables de entorno (tu GOOGLE_API_KEY desde el archivo .env)
load_dotenv()

True

In [452]:
# Credenciales de la Base de Datos ANTIGUA 
DB_OLD_USER = "root"
DB_OLD_PASS = "" # Sin contraseña
DB_OLD_HOST = "127.0.0.1"
DB_OLD_NAME = "sai_v2"

# Credenciales de la Base de Datos NUEVA 
DB_NEW_USER = "root"
DB_NEW_PASS = "" # Sin contraseña
DB_NEW_HOST = "127.0.0.1"
DB_NEW_NAME = "sai_v5"

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")

STATIC_CONTENT_STRUCTURE = {
    "en": {
        "special_tag": "Urgent Help",
        'description_heading'    : 'Description Heading',
        'description_text'       : 'Detailed Description',
        'summary_title'          : 'Summary Title',
        'summary_text'           : 'Summary Text',
        'help_mini_title'        : "Help People In need",
        'help_title'             : "Start Donating Today, Make The Difference",
        'help_btn_text'          : "DONATE TO THIS PROJECT >"
    },
    "es": {
        "special_tag": "Ayuda urgente",
        "description_heading": "Subtitulo por defecto",
        "description_text": "Texto de descripción por defecto",
        'summary_title': 'Titulo de resumen',
        "summary_text": "Texto de resumen",
        'help_mini_title': "Ayuda a personas en necesidad",
        'help_title': "Comienza a donar hoy, Haz la diferencia",
        'help_btn_text': "DONAR A ESTE PROYECTO >"
    }
}

STATIC_CONTENT_JSON = json.dumps(STATIC_CONTENT_STRUCTURE, ensure_ascii=False)

with open("promptproject2.txt", 'r', encoding='utf-8') as f:
    system_prompt = f.read()




In [ ]:
def fetch_new_gallery_data():
    engine = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    query = "SELECT id,title FROM projects" ## selecciona las imagenes de pages, esto para buscar el titulo que corresponde al id nuevo

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df




def fetch_old_gallery_data():
    engine = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME) ##creo un motor de busqueda
    query = "SELECT title,id FROM pages" ## selecciona las imagenes de pages, esto para buscar el id

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df


def fetch_old_images_data():
    engine = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME) ##creo un motor de busqueda
    query = "SELECT project_id,image FROM project_galleries" ## selecciona las imagenes de pages, esto para buscar el id

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

def fetch_new_images_data():
    engine = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    query = "SELECT ext FROM uphotos" ## selecciona las imagenes de pages, esto para buscar el id

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df

def insert_photos(id,ext):
    engine_new = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    
    insert_query = text("""
        INSERT INTO uphotos (
            modelid,ext, created_at, updated_at
        ) VALUES (
            :modelid,:ext, NOW(), NOW()
        )
    """)
    try:
        with engine_new.connect() as connection:
            # Correctly define the 'record' dictionary with parameter values
            record_data = {
             
                "modelid": id,
                "ext": ext,
                
            }
            connection.execute(insert_query, record_data)
            connection.commit()
            return True # Indicate success
    except Exception as e:
        print(f"Error inserting photo: {e}")
        # Optionally, you could log the error here using a proper logging framework
        return False # Indicate failure



def insert_photos(id,ext):
    engine_new = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    
    insert_query = text("""
        INSERT INTO uphotos (
            modelid,ext, created_at, updated_at
        ) VALUES (
            :modelid,:ext, NOW(), NOW()
        )
    """)
    try:
        with engine_new.connect() as connection:
            # conectara conlos datos que se tiene
            record_data = {
             
                "modelid": id,
                "ext": ext,
                
            }
            connection.execute(insert_query, record_data)
            connection.commit()
            return True # agrego el registro
    except Exception as e:
        print(f"Error inserting photo: {e}")
        # error
        return False # nada


def limpiar_vector(title):

    vectorAux=title.split(",") ##divido en una lista el string
    limpieza=vectorAux[0] ##selecciono el titulo en ingles
    envio=limpieza[8:-1] ##con esto quito la primera parte {"end"
    return envio
    
def extension(imagen):

    vectorAux=imagen.split("/") ##divido en una lista el string
    size=len(vectorAux)
    return vectorAux[size-1]
    





def create_db_engine(user, password, host, db_name):
    """Crea un motor de conexión SQLAlchemy."""
    # Codifica la contraseña si contiene caracteres especiales
    password_encoded = quote_plus(password)
    connection_string = f"mysql+mysqlconnector://{user}:{password_encoded}@{host}/{db_name}"
    return create_engine(connection_string)

def looktitle(id,query1):
    title='S'
    size=len(query1) ##se busca en la nase de datos vieja
    for i in range(0,size):
        
        if(query1.iloc[i,1]==id):
            title=query1.iloc[i,0]
            return title
            
        
def lookid(title,query):
    size=len(query)
    for i in range(0,size):
        titlen=limpiar_vector(query.iloc[i,1])
        if(titlen==title):
            id=query.iloc[i,0]
            return id

    
    

query1=fetch_old_gallery_data()
query2=fetch_old_images_data()
query3=fetch_new_gallery_data()
query4=fetch_new_images_data()

lista=[] ##listas anidadas o matriz de busqueda


tamano=len(query2) ##recorreremos la tabla de imagenes para verificar cada una

size4=len(query4)
for i in range(0,tamano):
    band=0
    lista2=[] ##vector dentro de la listas
    image=query2.iloc[i,1] 
    ext=extension(image) ##encontramos la extension del jpg
    ##print(ext)
    lista2.append(ext)
    ida=query2.iloc[i,0]
    title=looktitle(ida,query1) ##con esta encontramos el titulo
    lista2.append(title)
    id=lookid(title,query3) ##aqui encontramos el model_id que necesimos
    lista2.append(int(id))
    for j in range(0,size4):
        if(query4.iloc[j,0]==ext):
            band=1
    if band!=1:
        lista.append(lista2)

    



     



#for i in range(0,len(lista)):
    ##insertar valores obtenidos

    #print(lista[i][2],lista[i][0])
    ##insert_photos(lista[i][2],lista[i][0])





